In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import time
import commons
from utils import load_wav_to_torch
import utils
from data_utils import AudioSpecLoader, AudioSpecCollate
from models import SynthesizerTrn


from glob import glob
from scipy.io.wavfile import write
from mel_processing import mel_spectrogram_torch, spec_to_mel_torch, spectrogram_torch


IITP

In [3]:
hps = utils.get_hparams_from_file("./configs/iitp_base.json")
from scipy.io.wavfile import write

In [6]:
net_g = SynthesizerTrn(
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./checkpoint_dir/G_*.pth", net_g, None)

INFO:root:Loaded checkpoint '/home/mhjeong/tools/IITP_vocoder/cvae/logs/iitp_ss/G_1026000.pth' (iteration 900)


In [9]:
filenames=glob('./test_wav_dir/**/*.wav',recursive=True)
# filenames=filenames[1000:]
# print(filenames)
for filename in filenames:
    audio, sampling_rate = load_wav_to_torch(filename)
    audio_norm = audio / 32768.0
    audio_norm = audio_norm.unsqueeze(0)
    spec = spectrogram_torch(audio_norm, 1024,
            22050, 256, 1024,
            center=False)
#     print(filename)
    f_name=os.path.basename(filename)
    mel = spec_to_mel_torch(
        spec, 
        hps.data.filter_length, 
        hps.data.n_mel_channels, 
        hps.data.sampling_rate,
        hps.data.mel_fmin, 
        hps.data.mel_fmax)
#     print(mel.shape)
    with torch.no_grad():
        mel = mel.cuda()
        spec_lengths = torch.LongTensor([mel.size(2)]).cuda()
        sid = None
        start=time.time() 
        audio_ = net_g.infer(mel, spec_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
        end = time.time()
#         print((len(audio)/16000)/(end-start))
        
        ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=True))
        ipd.display(ipd.Audio(audio_, rate=hps.data.sampling_rate, normalize=True))
#         write('./generated_files/'+f_name, 22050, audio_)

KeyboardInterrupt: 